In [76]:
"""
BERT Score Evaluation for English to Hindi Translation by LLM
"""

'\nBERT Score Evaluation for English to Hindi Translation by LLM\n'

In [ ]:
#!pip install bert-score transformers torch

In [26]:
from bert_score import BERTScorer
from langchain.docstore.document import Document
from langchain_text_splitters import CharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

In [6]:
scorer = BERTScorer(lang="hi",  model_type="bert-base-multilingual-cased")

In [78]:
import time
start_time = time.time()
occupation = "politician"
target_collection_name = occupation + "_json_files"
embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDINGS_MODEL)
print("Target Collection Is: " + target_collection_name)
chroma_db_from_disk = Chroma(persist_directory=CHROMA_PERSIST_DIRECTORY,                             
                            collection_name=target_collection_name,
                            embedding_function=embeddings)
#results = chroma_db_from_disk.similarity_search("", k=100)
results = chroma_db_from_disk._collection.get(include=["documents", "metadatas", "embeddings"])
documents = [
    Document(page_content=doc, metadata=meta)
    for doc, meta in zip(results["documents"], results["metadatas"])
]
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.4f} seconds")
print("Number of Documents: "+  str(len(documents)))

Target Collection Is: politician_json_files
Execution time: 1.4099 seconds
Number of Documents: 85


In [79]:
documents[0]

Document(metadata={'first_name': 'Inoke', 'last_name': '', 'source': 'json_politician'}, page_content='["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\\n\\n### Instruction:\\nTranslate this English JSON File into a Hindi Language JSON File. \\n                Do not any other extra text or note after the conversion of JSON. \\n                Make sure everything in your response is in Hindi strictly and there are no additional NOTE afterwards\\n\\n### Input:\\n{\'_id\': \'Q58316\',\'sex or gender\': [\'male\'], \'instance of\': [\'human\'], \'occupation\': [\'diplomat\', \'politician\'], \'date of birth\': [{\'time\': \'+1948-06-16T00:00:00Z\', \'timezone\': 0, \'before\': 0, \'after\': 0, \'precision\': 11, \'calendarmodel\': \'http://www.wikidata.org/entity/Q1985727\'}], \'position held\': [\'Member of the House of Representatives of Fiji\', \'Member of th

In [80]:
import json
import ast
import time
from tqdm import tqdm
precision_vals_final = []
recall_vals_final = []
f1_vals_final = []
precision_keys_final = []
recall_keys_final = []
f1_keys_final = []
errors = 0
for res in tqdm(documents):
    document = res  
    content = ast.literal_eval(document.page_content)[0]  # Parse the string as a Python literal and get the first element
    response_start = content.index("### Response:")
    json_start = content.index("{", response_start)
    json_end = content.rindex("}")
    json_string = content[json_start:json_end+1]
    try:
        parsed_dict = ast.literal_eval(json_string)
        json_data = json.dumps(parsed_dict, ensure_ascii=False, indent=2)
        #print(json_data)
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing JSON: {e}")
        errors += 1
        continue
    candidate_val = [
    " ".join(str(value) if isinstance(value, list) else str(value) for value in parsed_dict[key])
        for key in parsed_dict
    ]
    candidate_key = [str(key) for key in parsed_dict.keys()]
    match = re.search(r'### Input:(.*?)(?=###|<|eot_id)', content, re.DOTALL)
    if match:
        input_content = match.group(1).strip() 
        try:
            parsed_dict = ast.literal_eval(input_content)  # Safely parse the Python-like input
            json_data = json.dumps(parsed_dict, ensure_ascii=False, indent=2)
            #print(json_data)
        except (SyntaxError, ValueError) as e:
            print(f"Error parsing input: {e}")
    else:
        continue
    reference_val = [
    " ".join(str(value) if isinstance(value, list) else str(value) for value in parsed_dict[key])
        for key in parsed_dict
    ]
    reference_key = [str(key) for key in parsed_dict.keys()]
    try:
        P, R, F1 = scorer.score(candidate_val, reference_val)
        P_Key, R_Key, F1_Key = scorer.score(candidate_key, reference_key)
    except:
        continue
    precision_vals_final.append(P.mean().item())
    recall_vals_final.append(R.mean().item())
    f1_vals_final.append(F1.mean().item())
    precision_keys_final.append(P_Key.mean().item())
    recall_keys_final.append(R_Key.mean().item())
    f1_keys_final.append(F1_Key.mean().item())

 13%|█▎        | 11/85 [00:00<00:04, 17.91it/s]

Error parsing JSON: ':' expected after dictionary key (<unknown>, line 1)


 22%|██▏       | 19/85 [00:01<00:03, 18.09it/s]

Error parsing JSON: invalid character '–' (U+2013) (<unknown>, line 1)
Error parsing JSON: '{' was never closed (<unknown>, line 1)
Error parsing JSON: '{' was never closed (<unknown>, line 1)


 35%|███▌      | 30/85 [00:01<00:02, 24.33it/s]

...

Error parsing JSON: '{' was never closed (<unknown>, line 1)
Error parsing JSON: '{' was never closed (<unknown>, line 1)


 94%|█████████▍| 80/85 [00:04<00:00, 15.57it/s]

Error parsing JSON: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<unknown>, line 1)


100%|██████████| 85/85 [00:04<00:00, 19.65it/s]

Error parsing JSON: '{' was never closed (<unknown>, line 1)
Error parsing JSON: '{' was never closed (<unknown>, line 1)
Error parsing JSON: '{' was never closed (<unknown>, line 1)


In [81]:
print(errors)

19


In [82]:
import statistics
p_vals_final = statistics.mean(precision_vals_final)
r_vals_final = statistics.mean(recall_vals_final)
f1_val_final = statistics.mean(f1_vals_final)    
p_keys_final = statistics.mean(precision_keys_final)
r_keys_final = statistics.mean(recall_keys_final)
f1_key_final = statistics.mean(f1_keys_final)
print("Precision For JSON Values conversion:" + str(p_vals_final))
print("Recall For JSON Values conversion:"+ str(r_vals_final))
print("F1 for JSON Values conversion:"+ str(f1_val_final))
print("Precision for JSON Keys conversion:"+ str(p_keys_final))
print("Recall for JSON Keys conversion:"+ str(r_keys_final))
print("F1 for JSON Keys conversion:" + str(f1_key_final))

Precision For JSON Values conversion:0.766576367205587
Recall For JSON Values conversion:0.8016621634877962
F1 for JSON Values conversion:0.7833567724145692
Precision for JSON Keys conversion:0.7527416327904011
Recall for JSON Keys conversion:0.7685230107143007
F1 for JSON Keys conversion:0.7601783234497597
